# Jeu de devinette : Père Fouras vs Laurent Jalabert

Dans ce notebook, nous allons simuler le duel légendaire entre le Père Fouras et Laurent Jalabert en utilisant Semantic Kernel avec des agents conversationnels.

In [10]:
# Bloc 1 - Installation et imports
%pip install semantic-kernel python-dotenv --quiet
import os
import logging
from dotenv import load_dotenv
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.agents.strategies import KernelFunctionTerminationStrategy
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import KernelArguments

# Configuration des logs
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.StreamHandler()]
)
logger = logging.getLogger('FortBoyard')

# Chargement des variables d'environnement
load_dotenv()


Note: you may need to restart the kernel to use updated packages.


True

## Configuration des agents

In [11]:
# Bloc 2 - Création du kernel
MOT_A_DEVINER = "anticonstitutionnellement"

def create_kernel():
    kernel = Kernel()
    kernel.add_service(OpenAIChatCompletion(
        service_id="openai",
        ai_model_id="gpt-4o-mini",
        api_key=os.getenv("OPENAI_API_KEY")
    ))
    return kernel


## Définition des prompts

In [12]:
# Bloc 3 - Prompts des agents
PERE_FOURAS_PROMPT = f"""
Tu es le Père Fouras de Fort Boyard. 
Tu dois faire deviner le mot '{MOT_A_DEVINER}'. 
Utilise des charades et réponses énigmatiques. 
Ne révèle jamais directement le mot !
"""

LAURENT_JALABERT_PROMPT = """
Tu es Laurent Jalabert. 
Tu dois deviner le mot en posant des questions fermées (Oui/Non).
Sois perspicace et stratégique dans tes questions.
"""

## Création des agents avec stratégies personnalisées

In [13]:
# Bloc 4 - Définition des agents
pere_fouras = ChatCompletionAgent(
    kernel=create_kernel(),
    service_id="openai",
    name="Pere_Fouras",
    instructions=PERE_FOURAS_PROMPT,
)

laurent_jalabert = ChatCompletionAgent(
    kernel=create_kernel(),
    service_id="openai",
    name="Laurent_Jalabert",
    instructions=LAURENT_JALABERT_PROMPT,
)

## Stratégie de terminaison personnalisée

In [14]:
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents.chat_message_content import ChatMessageContent

# Bloc 5 - Logique de terminaison
class FortBoyardTerminationStrategy(TerminationStrategy):
    """Arrête la partie si le mot est deviné"""
    
    async def should_terminate(
        self, 
        agent: ChatCompletionAgent, 
        history: list[ChatMessageContent], 
        cancellation_token = None
    ) -> bool:
        if not history:
            return False
        
        last_message = str(history[-1].content).lower()
        return MOT_A_DEVINER in last_message

## Configuration du groupe de discussion

In [24]:
# Bloc 6 - Configuration corrigée
chat = AgentGroupChat(
    agents=[pere_fouras, laurent_jalabert],
    termination_strategy=FortBoyardTerminationStrategy(
        agents=[laurent_jalabert],  # Définit explicitement les agents
        maximum_iterations=10       # Définit le nombre max d'itérations
    )
)


## Lancement de la partie !

In [28]:
from semantic_kernel.contents import AuthorRole, ChatMessageContent

# Bloc 7 - Exécution du jeu
async def jouer_partie():
    logger.info("🚀 Départ du duel Père Fouras vs Laurent Jalabert !")
    logger.info(f"Mot à deviner : {MOT_A_DEVINER.upper()}")
    
    # Ajout du message système directement dans l'historique du chat
    # await chat.add_chat_message(ChatMessageContent(role=AuthorRole.DEVELOPER, content="Nouvelle partie commencée !"))

    
    async for message in chat.invoke():
        role = message.role
        # name = message.author_name or "System"
        logger.info(f"[{role}] : {message.content}")
    
    logger.info("🎉 Partie terminée !")

await jouer_partie()


2025-02-07 11:55:55,549 [INFO] 🚀 Départ du duel Père Fouras vs Laurent Jalabert !
2025-02-07 11:55:55,550 [INFO] Mot à deviner : ANTICONSTITUTIONNELLEMENT
2025-02-07 11:55:55,551 [INFO] Selected agent at index 1 (ID: 71ef77c4-b750-4beb-aa27-2aea1cc559a5, name: Laurent_Jalabert)
2025-02-07 11:55:55,551 [INFO] Invoking agent Laurent_Jalabert
2025-02-07 11:55:56,531 [INFO] HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-02-07 11:55:56,533 [INFO] OpenAI usage: CompletionUsage(completion_tokens=15, prompt_tokens=399, total_tokens=414, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))
2025-02-07 11:55:56,534 [INFO] [ChatCompletionAgent] Invoked OpenAIChatCompletion with message count: 4.
2025-02-07 11:55:56,535 [INFO] [AuthorRole.ASSISTANT] : Est-ce que le mot désigne une notion morale